In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from random import randint
from matplotlib import pyplot as plt

In [ ]:
import datetime
from common import get_full_data
# load data
def steel_strength():
    df = pd.read_csv("dataset/steel_strength.csv")
    # get dependent and independent features
    X=df.iloc[:,1:-3]
    y=df.iloc[:,-2]
    return get_full_data(X,y)

def renewable():
    df = pd.read_csv("dataset/Renewable.csv")
    time = df["Time"].apply(lambda x: datetime.datetime.fromisoformat(x))
    df=df.drop(columns=["Time"])
    df["month"] = time.apply(lambda t: t.month)
    df["day"] = time.apply(lambda t: t.day)
    df["hour"] = time.apply(lambda t: t.hour)
    df["minute"] = time.apply(lambda t: t.minute)
    return df.iloc[:,1:], df.iloc[:,0]

In [ ]:
from xgboost import XGBRegressor
X,y = renewable()

# for high-dimensional data use `gpu` for device if you have one
special_model = XGBRegressor(device='cpu')

In [ ]:
from sklearn.decomposition import PCA

from render import *

pca = PCA(n_components=2)
X_small = pca.fit_transform(X)
y_n = y.to_numpy()[:,np.newaxis]
X_small=np.concatenate([X_small,np.ones_like(y_n),y_n],axis=1)
print(sum(pca.explained_variance_ratio_))
np.random.shuffle(X_small)
plot_2d_rgb(X_small,"original data",["d1","d2","generated power"], template=None)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from common import XGB_search_params

params = XGB_search_params()
state = randint(0,1000)
search = RandomizedSearchCV(
    special_model,
    params,
    n_iter=100,
    cv=5,
    random_state=state,
    n_jobs=-1,
)

# amount of samples used for parameters search
search_space_samples=5000

if search_space_samples>=len(X):
    search_space_samples=len(X)-1

_,X_search,_,y_search = train_test_split(X,y,test_size=search_space_samples/len(X))

search.fit(X_search,y_search)
special_model=search.best_estimator_

In [ ]:
# do repeated stratified k-fold cross-validation with classification report
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, cross_val_score
from common import cross_val_score_mean_std

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=50)
r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 scoring")
cross_val_score_mean_std(cross_val_score(special_model,X,y,cv=cv,scoring=r2_scoring),y.name)

In [ ]:
# New method
from common import find_outliers
X_numpy = X.to_numpy()
y_numpy = y.to_numpy()

outliers_mask, score = find_outliers(
    X_numpy,
    y_numpy,
    special_model,
    outliers_to_remove=0.1,
    iterations=5,
    gamma=0.99,
    evaluate_loss=metrics.mean_squared_error,
    cv=5,
    repeats=3,
    plot=True
)
print("removed ",np.sum(outliers_mask)/len(y))
X_clean = X_numpy[~outliers_mask]
y_clean = y_numpy[~outliers_mask]

r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 score")
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)


In [ ]:
X_clean = X_numpy[~outliers_mask]
y_clean = y_numpy[~outliers_mask][:,np.newaxis]

X_clean_small = pca.transform(X_clean)
to_render=np.concatenate([X_clean_small,np.ones_like(y_clean),y_clean],axis=1)
plot_2d_rgb(to_render,"clean data",["d1","d2","generated power"],template=None)

In [ ]:
# z-score method
from scipy import stats
data = pd.concat([X,y],axis=1)
z = np.abs(stats.zscore(data))
threshold = 3
data_clean = data[(z < threshold).all(axis=1)]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 score")
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

In [ ]:

import pandas as pd

# Assuming 'data' is a Pandas DataFrame
Q1 = data.quantile(0.05)
Q3 = data.quantile(0.95)
IQR = Q3 - Q1
data_clean = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

In [ ]:
from sklearn.ensemble import IsolationForest


clf = IsolationForest(random_state=50)
outliers_pred=clf.fit_predict(data)

data_clean = data[outliers_pred==1]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

Compare performance with some control model `KernelRidge` with `rbf` kernel and standard scaler

do parameters search for it using `RandomSearchCV`